## Instructions

<b>The general objective is to detect blue cursor, yellow timer, and human skin from input video. Your output should be similar to demo.mp4. Please complete steps 1-10 in one single code cell, step 11 in another markdown cell, and upload your Jupyter notebook file (*.ipynb). The whole process can be divided to the following steps:</b>

1. (5pts) Input images from video file WiiPlay.mp4 with the same level number as the last two digits of your student id, and show the images in the "input" window.

2. (5pts) Use <i>cv2.cvtColor()</i> to convert images from BGR to HSV format.

3. (10pts) Use <i>cv2.createTrackbar()</i> to create six trackbars (HueMin, HueMax, SatMin SatMax, ValMin, ValMax), and use <i>cv2.getTrackbarPos()</i> to get the current value of each trackbar.

4. (10pts) Use <i>cv2.threshold()</i> or <i>cv2.inRange()</i> to apply double thresholding to each channel (Hue, Sat, Val) based on current values of the six trackbars.

5. (10pts) Apply morphological filters to remove noise (outliers & holes), and show the detected regions in the "test" window.

6. (10pts) Find out the best color range to detect <b>blue cursor</b>, apply these thresholds, and show the detected regions in the "cursor" window.

7. (10pts) Find out the best color range to detect <b>yellow timer</b>, apply these thresholds, and show the detected regions in the "timer" window.

8. (10pts) Find out the best color range to detect <b>human skin</b>, apply these thresholds, and show the detected regions in the "skin" window.

9. (10pts) Use <i>cv2.connectedComponents()</i> and <i>cv2.putText()</i> to count and display how many skin regions in each frame.

10. (10pts) Show each individual skin region using different color.

11. (10pts) Which steps you believe you have completed? Which steps bother you?


<b>WiiPlay.mp4</b> input file click [here](https://drive.google.com/file/d/15XSgfwoPRxu48nUm49mNCqHVXlepLxLN/view?usp=share_link)
 

In [1]:
import cv2
import numpy as np
np.seterr(divide='ignore',invalid='ignore')

def hMaxCallback(pos):
    pass
def hMinCallback(pos):
    pass


cap = cv2.VideoCapture('WiiPlay.mp4')
# Check if the video file is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open the video file")



""" 1. Input images from video file WiiPlay.mp4 with the same level number as the last two digits of your student id. """

# Each video has different time length and frame rate.
# frame_seq = start_time * fps
# time_length = seconds * fps
time_length = 900.0
fps = 30
frame_seq = 2940
out_size = (640, 360)

cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq);



""" Show the images in the "input" window. """

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    cur_frame = cv2.resize(frame, out_size, 0, 0, interpolation=cv2.INTER_AREA)
    cv2.imshow('Input', cur_frame)



    """ 2. Use cv2.cvtColor() to convert images from BGR to HSV format. """

    hsv_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2HSV)
    cv2.imshow('HSV', hsv_frame)



    """ 3. Use cv2.createTrackbar() to create six trackbars: (HueMin, HueMax, SatMin SatMax, ValMin, ValMax) """

    cv2.namedWindow('Test')

    cv2.createTrackbar('HueMin', 'Test',    0, 179, hMinCallback)
    cv2.createTrackbar('HueMax', 'Test',  179, 179, hMaxCallback) 
    cv2.createTrackbar('SatMin', 'Test',    0, 255, hMinCallback)
    cv2.createTrackbar('SatMax', 'Test',  255, 255, hMaxCallback) 
    cv2.createTrackbar('ValMin', 'Test',    0, 255, hMinCallback)
    cv2.createTrackbar('ValMax', 'Test',  255, 255, hMaxCallback) 



    """ Use cv2.getTrackbarPos() to get the current value of each trackbar. """

    hMax = cv2.getTrackbarPos('HueMax','Test')
    hMin = cv2.getTrackbarPos('HueMin','Test')
    sMax = cv2.getTrackbarPos('SatMax','Test')
    sMin = cv2.getTrackbarPos('SatMin','Test')
    vMax = cv2.getTrackbarPos('ValMax','Test')
    vMin = cv2.getTrackbarPos('ValMin','Test')



    """ 4. Use cv2.threshold() or cv2.inRange() to apply double thresholding to each channel (Hue, Sat, Val) """
    """ Based on current values of the six trackbars """

    upper = np.array([hMax, sMax, vMax])
    lower = np.array([hMin, sMin, vMin])
    
    mask = cv2.inRange(cur_frame, lower, upper)
    # masked_frame = cur_frame
    # masked_frame[mask == 0] = 0
    # cv2.imshow('Test', masked_frame)
    # cv2.imshow('Test', mask)
    


    """ 5. Apply morphological filters to remove noise (outliers & holes). """

    kernel = np.ones((5, 5), np.uint8)
    
    # img_erosion = cv2.erode(cur_frame, kernel, iterations=1)
    # img_dilation = cv2.dilate(cur_frame, kernel, iterations=1)
    img_opening = cv2.morphologyEx(cur_frame, cv2.MORPH_OPEN, kernel)
    # img_closing = cv2.morphologyEx(cur_frame, cv2.MORPH_CLOSE, kernel)

    masked_frame = cv2.inRange(img_opening, lower, upper)

    """ Show the detected regions in the "test" window. """
    
    cv2.imshow('Test', masked_frame)
    

     
    """ 6. Find out the best color range to detect blue cursor, apply these thresholds """
    
    cursor_lower = np.array([100,100,100]) 
    cursor_upper = np.array([124, 255, 255]) 
    cursor_mask = cv2.inRange(hsv_frame,cursor_lower,cursor_upper) 

    """ Show the detected regions in the "cursor" window. """

    cv2.imshow('Cursor',cursor_mask)
    

    
    """ 7. Find out the best color range to detect yellow timer, apply these thresholds """ 
    
    timer_lower = np.array([22,80,70]) 
    timer_upper = np.array([34, 255, 255]) 
    timer_mask = cv2.inRange(hsv_frame,timer_lower,timer_upper) 

    """ Show the detected regions in the "timer" window. """

    cv2.imshow('Timer',timer_mask)


    
    """ 8. Find out the best color range to detect human skin, apply these thresholds """
    
    skin_lower = np.array([10,28,100]) 
    skin_upper = np.array([16, 255, 255]) 
    skin_mask = cv2.inRange(hsv_frame,skin_lower,skin_upper) 

    """ Show the detected regions in the "skin" window. """

    cv2.imshow('Skin',skin_mask)
    
    

    """ 9. Use cv2.connectedComponents() and cv2.putText() to count and display how many skin regions in each frame. """
    
    _, skin_maskDisplay = cv2.threshold(skin_mask, 200, 255, cv2.THRESH_BINARY)
    _, bin_img = cv2.threshold(skin_mask, 200, 255, cv2.THRESH_BINARY_INV)
    
    _, labels = cv2.connectedComponents(skin_maskDisplay)
    ret, _ = cv2.connectedComponents(bin_img)
    
    # Noise adjustments
    ret = ret - 2
    # print(ret)
        
    """ 10. Show each individual skin region using different color. """
    
    # Hue controls the color
    label_hue = np.uint8(179 * labels / np.max(labels))
    
    # Saturation and Value set to same size blank array
    blank_channel = 255 * np.ones_like(label_hue)
    
    # Merge HSV
    labeled_img = cv2.merge([label_hue, blank_channel, blank_channel])

    # Convert to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # Set bg(background) label to black
    labeled_img[label_hue == 0] = 0
    
    # Setup text (text must be string)
    regions = str(ret)
    WHITE_BGR = (255, 255, 255)
    cv2.putText(labeled_img, regions , (0,300), cv2.FONT_HERSHEY_SIMPLEX, 2, WHITE_BGR , 4, cv2.LINE_AA)
    
    cv2.imshow('Count and Display',labeled_img)
    


    # stop til time_length = 0 
    time_length = time_length - 1
    if time_length == 0:
        break
    c = cv2.waitKey(1)
    if c == 32:
        cv2.waitKey()
    if c == 27:
        break


cap.release()
cv2.destroyAllWindows()

-1

11. (10pts) Which steps you believe you have completed? Which steps bother you?

I believe I completed all the steps. The most bothering step for me among all is the skin mask display. I use the inRange function to apply double thresholding, calculate the range of skin color, but it detects other colors which are in the same range of human skin. This effects the skin amount with some noise.